In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
movies = pd.read_csv('movies.csv') #import fill movies
ratings = pd.read_csv('ratings.csv') #import file rating
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
#create column year
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies['title'] = movies['title'].apply(lambda x: x.strip())

#use One Hot Encoding to encode genre, drop column genres
movies['genres'] = movies.genres.str.split('|')
for i, row in movies.iterrows():
    for genre in row['genres']:
        movies.at[i, genre] = 1
movies = movies.drop('genres', axis = 1).drop('year', axis = 1)
movies = movies.fillna(0) 

movies.head()

/var/folders/hw/t1pd79hx7f36fky47_32wvtw0000gn/T/ipykernel_70475/3989036808.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [30]:
ratings = ratings.drop('timestamp', axis = 1)
ratings.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


    Content based

In [31]:
userRatingInput = [
            {'title':'Toy Story', 'rating':3},
            {'title':'Jumanji', 'rating':4},
            {'title':'Grumpier Old Men', 'rating':5},
            {'title':'Waiting to Exhale', 'rating':5},
            {'title':'Father of the Bride Part II', 'rating':4.5}] 
user_rating_input = pd.DataFrame(userRatingInput)
user_rating_input

,title,rating
0,Toy Story,3.0
1,Jumanji,4.0
2,Grumpier Old Men,5.0
3,Waiting to Exhale,5.0
4,Father of the Bride Part II,4.5


In [32]:
#create table user movies input
inputMovies = movies[movies['title'].isin(user_rating_input['title'].tolist())]
userMoviesInput = pd.merge(inputMovies, user_rating_input)
userMoviesInput
#create user movies matrix
userGenreMoviesMatrix = userMoviesInput.drop('movieId', axis = 1).drop('title', axis = 1).drop('rating', axis = 1)
userGenreMoviesMatrix = userGenreMoviesMatrix.transpose()
userGenreMoviesMatrix
#create user profile
userProfile = userGenreMoviesMatrix.dot(user_rating_input['rating'])
userProfile

Adventure              7.0
Animation              3.0
Children               7.0
Comedy                17.5
Fantasy                7.0
Romance               10.0
Drama                  5.0
Action                 0.0
Crime                  0.0
Thriller               0.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 0.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [49]:
#create movies matrix
genreMovies = movies.set_index(movies['movieId'])
genreMovies = genreMovies.drop('movieId', axis = 1).drop('title', axis = 1)
#create recommendation matrix and normalization
recommendationData = (genreMovies*userProfile).sum(axis=1)/(userProfile.sum())
recommendationData = recommendationData.sort_values(ascending=False)
#select top 10
recommendationData

movieId
26093     1.000000
92348     0.911504
56152     0.911504
84637     0.911504
4306      0.911504
            ...   
104636    0.000000
104642    0.000000
104664    0.000000
104667    0.000000
151711    0.000000
Length: 34208, dtype: float64

In [58]:
recommendationDataFinal = recommendationData.head(10)
recommendationInfo = movies.loc[movies['movieId'].isin(recommendationDataFinal.head(10).keys())]
recommendationInfo = recommendationInfo[['movieId','title']].reset_index(drop=True)
recommendationInfo

,movieId,title
0,1907,Mulan
1,4306,Shrek
2,26093,"Wonderful World of the Brothers Grimm, The"
3,27344,Revolutionary Girl Utena: Adolescence of Utena...
4,56152,Enchanted
5,84637,Gnomeo & Juliet
6,92348,Puss in Boots (Nagagutsu o haita neko)
7,108540,Ernest & Célestine (Ernest et Célestine)
8,121249,The Magical Legend of the Leprechauns
9,146305,Princes and Princesses
